In [15]:
from youtube_transcript_api import YouTubeTranscriptApi


In [16]:
def get_youtube_transcript(video_url: str) -> list:
    video_id = video_url.split("v=")[1]
    transcript = YouTubeTranscriptApi.get_transcript(video_id)
    return transcript
video_url = "https://www.youtube.com/watch?v=8QfI5a7lTKU"
video_id = video_url.split("v=")[1]

transcript = YouTubeTranscriptApi.get_transcript(video_id)

for segment in transcript:
    print(segment['text'])

what is the common point between all
those robots
well all the robots you see here are
or can be powered by rouse
roth means robot operating system
it is something between a middleware
and a framework built especially for
robotics applications rao's goal
is to provide a standard for robotic
software
that developers can use and reuse in
any robot once you know how to use ros
efficiently you will be able to set up
a new software for a new robot in no
time
you first need to master the basics and
core functionalities
which you will see in this course and
then
for every new robot you program you will
gain more skills
that can apply to other robots
the more you know about ross the easiest
it will be for you to program and
understand
the code of any robot
but is that it no another
philosophy behind ross is don't reinvent
the wheel reinventing the wheel is a
very common problem
in robotics people always start building
new software
that is already existing and thus they
don't have the time
to w

In [17]:
## MongoDB
from pymongo import MongoClient

# MongoDB connection
client = MongoClient("mongodb://localhost:27017/")  # Replace with your MongoDB connection string
db = client["youtube_database"]  # Database name
collection = db["transcripts"]   # Collection name

# Insert into MongoDB
document = {
    "source": "youtube",
    "video_id": video_id,
    "transcript": transcript
}

collection.insert_one(document)
print("Transcript stored into MongoDB")

Transcript stored into MongoDB


In [18]:
result = collection.find_one({"video_id": video_id})
print(result.keys())

dict_keys(['_id', 'source', 'video_id', 'transcript'])


In [19]:
# Extract text from transcripts
sentences = [i['text'] for i in result['transcript']]
print(sentences)







['what is the common point between all', 'those robots', 'well all the robots you see here are', 'or can be powered by rouse', 'roth means robot operating system', 'it is something between a middleware', 'and a framework built especially for', "robotics applications rao's goal", 'is to provide a standard for robotic', 'software', 'that developers can use and reuse in', 'any robot once you know how to use ros', 'efficiently you will be able to set up', 'a new software for a new robot in no', 'time', 'you first need to master the basics and', 'core functionalities', 'which you will see in this course and', 'then', 'for every new robot you program you will', 'gain more skills', 'that can apply to other robots', 'the more you know about ross the easiest', 'it will be for you to program and', 'understand', 'the code of any robot', 'but is that it no another', "philosophy behind ross is don't reinvent", 'the wheel reinventing the wheel is a', 'very common problem', 'in robotics people always

In [20]:
# Chunk sentences together for embedding
#TODO: Make this chunking better:Maybe capitalize and fins end of sentences in a better way
sentences_in_chunk = 6
chunks = []
for i in range(0, len(sentences), sentences_in_chunk):
    chunks.append(". ".join(sentences[i:i + sentences_in_chunk]))

print(len(chunks))

28


In [21]:
# Embed these chunks
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2') # TODO: Pick better models

embeddings = model.encode(chunks)
embeddings_dict = dict(zip(chunks, embeddings))
print(embeddings.shape)


(28, 384)


In [22]:
# Store embeddings in MongoDB
# Insert into MongoDB
collection = db["embeddings"]   # Collection name
document = {
    "source": "youtube",
    "video_id": video_id,
    "embeddings": embeddings.tolist()
}

collection.insert_one(document)
print("Embeddings stored into MongoDB")


Embeddings stored into MongoDB


In [23]:
# TODO: Do further preprocessing to tokens

In [24]:
## Similarity search using qdrant
from qdrant_client import QdrantClient
from qdrant_client.http import models
from torchvision.transforms import Resize
from tqdm import tqdm
import torch

qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance, for testing, CI/CD

#Create a collection or database of texts where you store the embeddings
my_collection = "text_collection"

qdrant.create_collection(
    collection_name=my_collection,
    vectors_config=models.VectorParams(size=384, distance=models.Distance.COSINE)
)

# Insert embeddings into Qdrant
for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
    qdrant.upsert(
        collection_name=my_collection,
        points=[models.PointStruct(
            id=i,
            vector=embedding,
            payload={"text": chunk}
        )]
    )






In [25]:
qdrant.count(
    collection_name=my_collection,
    exact=True,
)

CountResult(count=28)

In [26]:
## Sanity test
sample_sentence = "What is the meaning of life?"
sample_embedding = model.encode(sample_sentence)
search_result = qdrant.search(
    collection_name=my_collection,
    query_vector=sample_embedding,
    limit=5
)

In [27]:
search_result

[ScoredPoint(id=21, version=0, score=0.1516602337360382, payload={'text': "and great libraries that's not all. ros is said to be language agnostic. it means that you can program some parts. of your application. in a programming language and another. part"}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=10, version=0, score=0.07326249778270721, payload={'text': 'create a robot software. which needs a lot of communication. between its. sub-programs or has some functionalities. that go beyond a very simple use case. etc so for a robot'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=0, version=0, score=0.06109253689646721, payload={'text': 'what is the common point between all. those robots. well all the robots you see here are. or can be powered by rouse. roth means robot operating system. it is something between a middleware'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=22, version=0, score=0.05160970240831375, payload={'text': "i

# LLM

In [28]:
# Get GPU available memory
import torch
gpu_memory_bytes = torch.cuda.get_device_properties(0).total_memory
gpu_memory_gb = round(gpu_memory_bytes / (2**30))
print(f"Available GPU memory: {gpu_memory_gb} GB")

Available GPU memory: 24 GB


In [29]:
from huggingface_hub import login
login()

In [30]:
use_quantization_config = True
model_id = "TheBloke/CapybaraHermes-2.5-Mistral-7B-GPTQ"

In [31]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.utils import is_flash_attn_2_available 

# 1. Create quantization config for smaller model loading (optional)
# Requires !pip install bitsandbytes accelerate, see: https://github.com/TimDettmers/bitsandbytes, https://huggingface.co/docs/accelerate/
# For models that require 4-bit quantization (use this if you have low GPU memory available)
from transformers import BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.float16)

# Bonus: Setup Flash Attention 2 for faster inference, default to "sdpa" or "scaled dot product attention" if it's not available
# Flash Attention 2 requires NVIDIA GPU compute capability of 8.0 or above, see: https://developer.nvidia.com/cuda-gpus
# Requires !pip install flash-attn, see: https://github.com/Dao-AILab/flash-attention 
if (is_flash_attn_2_available()) and (torch.cuda.get_device_capability(0)[0] >= 8):
  attn_implementation = "flash_attention_2"
else:
  attn_implementation = "sdpa"
print(f"[INFO] Using attention implementation: {attn_implementation}")

# 2. Pick a model we'd like to use (this will depend on how much GPU memory you have available)
# model_id = "google/gemma-7b-it"
model_id = model_id # (we already set this above)
print(f"[INFO] Using model_id: {model_id}")

# 3. Instantiate tokenizer (tokenizer turns text into numbers ready for the model) 
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

# 4. Instantiate the model
llm_model = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")
# llm_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_id)

if not use_quantization_config: # quantization takes care of device setting automatically, so if it's not used, send model to GPU 
    llm_model.to("cuda")

[INFO] Using attention implementation: sdpa
[INFO] Using model_id: TheBloke/CapybaraHermes-2.5-Mistral-7B-GPTQ


CUDA extension not installed.
CUDA extension not installed.
c:\Users\harsh\anaconda3\envs\ai\lib\site-packages\transformers\modeling_utils.py:5006: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Some weights of the model checkpoint at TheBloke/CapybaraHermes-2.5-Mistral-7B-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_att

In [32]:
llm_model.eval() # put model in evaluation mode
llm_model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32002, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (rotary_emb): MistralRotaryEmbedding()
          (k_proj): QuantLinear()
          (o_proj): QuantLinear()
          (q_proj): QuantLinear()
          (v_proj): QuantLinear()
        )
        (mlp): MistralMLP(
          (act_fn): SiLU()
          (down_proj): QuantLinear()
          (gate_proj): QuantLinear()
          (up_proj): QuantLinear()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,), eps=1e-05)
  )
  (lm_head): Linear(in_features=4096, out_features=32002, bias=False)
)

In [33]:
def get_model_num_params(model: torch.nn.Module):
    return sum([param.numel() for param in model.parameters()])

get_model_num_params(llm_model)

262426624

In [34]:
def get_model_mem_size(model: torch.nn.Module):
    """
    Get how much memory a PyTorch model takes up.

    See: https://discuss.pytorch.org/t/gpu-memory-that-model-uses/56822
    """
    # Get model parameters and buffer sizes
    mem_params = sum([param.nelement() * param.element_size() for param in model.parameters()])
    mem_buffers = sum([buf.nelement() * buf.element_size() for buf in model.buffers()])

    # Calculate various model sizes
    model_mem_bytes = mem_params + mem_buffers # in bytes
    model_mem_mb = model_mem_bytes / (1024**2) # in megabytes
    model_mem_gb = model_mem_bytes / (1024**3) # in gigabytes

    return {"model_mem_bytes": model_mem_bytes,
            "model_mem_mb": round(model_mem_mb, 2),
            "model_mem_gb": round(model_mem_gb, 2)}

get_model_mem_size(llm_model)

{'model_mem_bytes': 4155817984, 'model_mem_mb': 3963.3, 'model_mem_gb': 3.87}

In [35]:
input_text = "What is ros?"
print(f"Input text:\n{input_text}")

system_message = "You are a robotic operating system (ROS) developer"

# Create prompt template for instruction-tuned model
dialogue_template = [
    {"role": "user",
     "content": input_text}
]

prompt=f'''<|im_start|>system
{system_message}<|im_end|>
<|im_start|>user
{input_text}<|im_end|>
<|im_start|>assistant
'''

# Apply the chat template
# prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
#                                        tokenize=False, # keep as raw text (not tokenized)
#                                        add_generation_prompt=True)
# print(f"\nPrompt (formatted):\n{prompt}")

# # Tokenize the input text
# input_ids = tokenizer(input_text, return_tensors="pt").input_ids
# print(f"Input IDs:\n{input_ids}")

# prompt = input_text #TODO: Fix a prompt structure?

Input text:
What is ros?


In [36]:
prompt

'<|im_start|>system\nYou are a robotic operating system (ROS) developer<|im_end|>\n<|im_start|>user\nWhat is ros?<|im_end|>\n<|im_start|>assistant\n'

In [37]:
# Tokenize the input text (turn it into numbers) and send it to GPU
input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
print(f"Model input (tokenized):\n{input_ids}\n")

# Generate outputs passed on the tokenized input
# See generate docs: https://huggingface.co/docs/transformers/v4.38.2/en/main_classes/text_generation#transformers.GenerationConfig 
outputs = llm_model.generate(**input_ids, 
                             temperature=0.7, 
                             do_sample=True, 
                             top_p=0.95, 
                             top_k=40, 
                             max_new_tokens=512) # define the maximum number of new tokens to create
# outputs = llm_model.generate(**input_ids,
#                              max_new_tokens=256) # define the maximum number of new tokens to create
print(f"Model output (tokens):\n{outputs[0]}\n")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Model input (tokenized):
{'input_ids': tensor([[    1, 32001,  1587,    13,  1976,   460,   264,  7006,  7839, 10513,
          1587,   325,  1594, 28735, 28731, 21782, 32000, 28705,    13, 32001,
          2188,    13,  3195,   349,   712, 28713, 28804, 32000, 28705,    13,
         32001, 13892,    13]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

Model output (tokens):
tensor([    1, 32001,  1587,    13,  1976,   460,   264,  7006,  7839, 10513,
         1587,   325,  1594, 28735, 28731, 21782, 32000, 28705,    13, 32001,
         2188,    13,  3195,   349,   712, 28713, 28804, 32000, 28705,    13,
        32001, 13892,    13,  1594, 28735, 28725,   442,  5776,   322,  6692,
         1077,  2135, 28725,   349,   396,  1565, 28733,  1394, 28725, 11678,
        28733,  3015,  1077,  1587, 13395,  5682,   354,   272,  4099,   304,
        12364,   302, 18401

In [38]:
# Decode the output tokens to text
outputs_decoded = tokenizer.decode(outputs[0])
print(f"Model output (decoded):\n{outputs_decoded}\n")

Model output (decoded):
<s><|im_start|> system
You are a robotic operating system (ROS) developer<|im_end|> 
<|im_start|> user
What is ros?<|im_end|> 
<|im_start|> assistant
ROS, or Robot Operating System, is an open-source, meta-operating system primarily designed for the development and maintenance of robotics applications. It provides a flexible, modular framework to develop, integrate, and maintain various components of a robot's software stack, such as perception, localization, motion planning, and decision-making.

ROS consists of a set of tools, libraries, and conventions that facilitate the creation of complex, modular robotics software systems. It allows developers to easily reuse code, integrate new algorithms, and collaborate on robotics projects.

Key features of ROS include a message-passing system for inter-process communication, a package management system, a set of tools for building and running robot applications, and a large collection of pre-built packages for variou

In [39]:
outputs_decoded

"<s><|im_start|> system\nYou are a robotic operating system (ROS) developer<|im_end|> \n<|im_start|> user\nWhat is ros?<|im_end|> \n<|im_start|> assistant\nROS, or Robot Operating System, is an open-source, meta-operating system primarily designed for the development and maintenance of robotics applications. It provides a flexible, modular framework to develop, integrate, and maintain various components of a robot's software stack, such as perception, localization, motion planning, and decision-making.\n\nROS consists of a set of tools, libraries, and conventions that facilitate the creation of complex, modular robotics software systems. It allows developers to easily reuse code, integrate new algorithms, and collaborate on robotics projects.\n\nKey features of ROS include a message-passing system for inter-process communication, a package management system, a set of tools for building and running robot applications, and a large collection of pre-built packages for various robotics tas

In [40]:
a = prompt.replace('<|im_start|>', '<|im_start|> ').replace('<|im_end|>\n', '<|im_end|> \n')


In [41]:
a

'<|im_start|> system\nYou are a robotic operating system (ROS) developer<|im_end|> \n<|im_start|> user\nWhat is ros?<|im_end|> \n<|im_start|> assistant\n'

In [42]:
for i in range(len(a)):
    if not a[i] == outputs_decoded[3 + i]:
        print(i, a[i], outputs_decoded[3 + i])

In [43]:
outputs_decoded.replace(a, '')

"<s>ROS, or Robot Operating System, is an open-source, meta-operating system primarily designed for the development and maintenance of robotics applications. It provides a flexible, modular framework to develop, integrate, and maintain various components of a robot's software stack, such as perception, localization, motion planning, and decision-making.\n\nROS consists of a set of tools, libraries, and conventions that facilitate the creation of complex, modular robotics software systems. It allows developers to easily reuse code, integrate new algorithms, and collaborate on robotics projects.\n\nKey features of ROS include a message-passing system for inter-process communication, a package management system, a set of tools for building and running robot applications, and a large collection of pre-built packages for various robotics tasks.\n\nIn summary, ROS is a powerful, flexible, and open-source tool that simplifies the development and maintenance of robotics software applications, 

In [44]:
# print(prompt)
print(f"Model output (decoded):\n{outputs_decoded.replace('<s>', '').replace(a, '').replace('<|im_end|>', '')}\n")

Model output (decoded):
ROS, or Robot Operating System, is an open-source, meta-operating system primarily designed for the development and maintenance of robotics applications. It provides a flexible, modular framework to develop, integrate, and maintain various components of a robot's software stack, such as perception, localization, motion planning, and decision-making.

ROS consists of a set of tools, libraries, and conventions that facilitate the creation of complex, modular robotics software systems. It allows developers to easily reuse code, integrate new algorithms, and collaborate on robotics projects.

Key features of ROS include a message-passing system for inter-process communication, a package management system, a set of tools for building and running robot applications, and a large collection of pre-built packages for various robotics tasks.

In summary, ROS is a powerful, flexible, and open-source tool that simplifies the development and maintenance of robotics software 

In [45]:
print(f"Input text: {input_text}\n")
print(f"Output text:\n{outputs_decoded.replace('<s>', '').replace(a, '').replace('<|im_end|>', '')}")

Input text: What is ros?

Output text:
ROS, or Robot Operating System, is an open-source, meta-operating system primarily designed for the development and maintenance of robotics applications. It provides a flexible, modular framework to develop, integrate, and maintain various components of a robot's software stack, such as perception, localization, motion planning, and decision-making.

ROS consists of a set of tools, libraries, and conventions that facilitate the creation of complex, modular robotics software systems. It allows developers to easily reuse code, integrate new algorithms, and collaborate on robotics projects.

Key features of ROS include a message-passing system for inter-process communication, a package management system, a set of tools for building and running robot applications, and a large collection of pre-built packages for various robotics tasks.

In summary, ROS is a powerful, flexible, and open-source tool that simplifies the development and maintenance of rob

In [46]:
# Nutrition-style questions generated with GPT4
gpt4_questions = [
    "WWhat are the main differences between ROS 1 and ROS 2?",
    "Explain the concept of topics, services, and actions in ROS.",
    "What is the purpose of the catkin_make command in ROS 1?",
    "What is the function of rviz in ROS?",
    "What is the purpose of rqt_graph, and how can it help in debugging a ROS system?"
] 

# Manually created question list
manual_questions = [
    "What is ros?",
]

query_list = gpt4_questions + manual_questions

In [47]:
import random
query = random.choice(query_list)

print(f"Query: {query}")



Query: What is the function of rviz in ROS?


In [48]:
search_result = qdrant.search(
    collection_name=my_collection,
    query_vector=model.encode(query),
    limit=5
)

In [49]:
search_result[0].payload['text']

"and great libraries that's not all. ros is said to be language agnostic. it means that you can program some parts. of your application. in a programming language and another. part"

## Augment Similarity Search and LLM output 

In [50]:
def prompt_formatter(query: str, 
                     context_items: list[dict]) -> str:
    """
    Augments query with text-based context from context_items.
    """
    # Join context items into one dotted paragraph
    context = "- " + "\n- ".join([item["sentence_chunk"] for item in context_items])

    # # Create a base prompt with examples to help the model
    # # Note: this is very customizable, I've chosen to use 3 examples of the answer style we'd like.
    # # We could also write this in a txt file and import it in if we wanted.
    # base_prompt = """Based on the following context items, please answer the query.
    # Give yourself room to think by extracting relevant passages from the context before answering the query.
    # Don't return the thinking, only return the answer.
    # Make sure your answers are as explanatory as possible.
    # Use the following examples as reference for the ideal answer style.
    # \nExample 1:
    # Query: What are the fat-soluble vitamins?
    # Answer: The fat-soluble vitamins include Vitamin A, Vitamin D, Vitamin E, and Vitamin K. These vitamins are absorbed along with fats in the diet and can be stored in the body's fatty tissue and liver for later use. Vitamin A is important for vision, immune function, and skin health. Vitamin D plays a critical role in calcium absorption and bone health. Vitamin E acts as an antioxidant, protecting cells from damage. Vitamin K is essential for blood clotting and bone metabolism.
    # \nExample 2:
    # Query: What are the causes of type 2 diabetes?
    # Answer: Type 2 diabetes is often associated with overnutrition, particularly the overconsumption of calories leading to obesity. Factors include a diet high in refined sugars and saturated fats, which can lead to insulin resistance, a condition where the body's cells do not respond effectively to insulin. Over time, the pancreas cannot produce enough insulin to manage blood sugar levels, resulting in type 2 diabetes. Additionally, excessive caloric intake without sufficient physical activity exacerbates the risk by promoting weight gain and fat accumulation, particularly around the abdomen, further contributing to insulin resistance.
    # \nExample 3:
    # Query: What is the importance of hydration for physical performance?
    # Answer: Hydration is crucial for physical performance because water plays key roles in maintaining blood volume, regulating body temperature, and ensuring the transport of nutrients and oxygen to cells. Adequate hydration is essential for optimal muscle function, endurance, and recovery. Dehydration can lead to decreased performance, fatigue, and increased risk of heat-related illnesses, such as heat stroke. Drinking sufficient water before, during, and after exercise helps ensure peak physical performance and recovery.
    # \nNow use the following context items to answer the user query:
    # {context}
    # \nRelevant passages: <extract relevant passages from the context here>
    # User query: {query}
    # Answer:"""

    # # Update base prompt with context items and query   
    # base_prompt = base_prompt.format(context=context, query=query)

    # # Create prompt template for instruction-tuned model
    # dialogue_template = [
    #     {"role": "user",
    #     "content": base_prompt}
    # ]

    # # Apply the chat template
    # prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
    #                                       tokenize=False,
    #                                       add_generation_prompt=True)
    # return prompt

    system_message = f"You are a robotic operating system (ROS) developer, using given context as additional information and answer the query, just your answer explanatory answer would suffice. Here is the context:{context}." # For out task we can use this as a system message

    prompt=f'''<|im_start|>system
    {system_message}<|im_end|>
    <|im_start|>user
    {query}<|im_end|>
    <|im_start|>assistant
    '''
    return prompt

    
    

In [51]:
# import random
query = random.choice(query_list)
print(f"Query: {query}")

# Get relevant resources
# scores, indices = retrieve_relevant_resources(query=query,
#                                               embeddings=embeddings)
search_results = qdrant.search(
    collection_name=my_collection,
    query_vector=model.encode(query),
    limit=5
)
    
# Create a list of context items
# context_items = [pages_and_chunks[i] for i in indices]
context_items = []
for result in search_results:
    context_items.append({"sentence_chunk": result.payload['text']})

# Format prompt with context items
prompt = prompt_formatter(query=query,
                          context_items=context_items)
prompt

Query: What is the function of rviz in ROS?


"<|im_start|>system\n    You are a robotic operating system (ROS) developer, using given context as additional information and answer the query, just your answer explanatory answer would suffice. Here is the context:- and great libraries that's not all. ros is said to be language agnostic. it means that you can program some parts. of your application. in a programming language and another. part\n- powered by ros. using python and c plus plus. ros is also open source with an. active and growing community so. you can directly see what's going on you. can get help\n- and a framework built especially for. robotics applications rao's goal. is to provide a standard for robotic. software. that developers can use and reuse in. any robot once you know how to use ros\n- so this is the why behind ross. now when to use ros maybe. you've already started to program robots. with. an arduino board with a custom code. on a computer and what you will quickly\n- all the external libraries that exist. for

In [52]:
input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

# Generate an output of tokens
outputs = llm_model.generate(**input_ids,
                             temperature=0.7, # lower temperature = more deterministic outputs, higher temperature = more creative outputs
                             do_sample=True, # whether or not to use sampling, see https://huyenchip.com/2024/01/16/sampling.html for more
                             top_p=0.95, 
                             top_k=40, 
                             max_new_tokens=512) # how many new tokens to generate from prompt 

# Turn the output tokens into text
output_text = tokenizer.decode(outputs[0])

print(f"Query: {query}")
a = prompt.replace('<|im_start|>', '<|im_start|> ').replace('<|im_end|>\n', '<|im_end|> \n')
print(f"RAG answer:\n{output_text.replace('<s>', '').replace(a, '').replace('<|im_end|>', '')}")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Query: What is the function of rviz in ROS?
RAG answer:

    Rviz, in ROS (Robotic Operating System), is a 3D visualization tool that serves as a visualization and interaction environment for ROS data. It allows users to visualize and interact with data from robots or simulations in a 3D environment.

    The primary functions of Rviz include:

    1. Visualization: Rviz can display various types of data, such as sensor data (e.g., from lidar or cameras), robot models, and path planning visualizations. This visualization helps users understand the robot's state and plan actions accordingly.
    2. Interaction: Rviz allows users to interact with the robot's environment by manipulating objects, setting waypoints, and controlling robot movement.
    3. Configuration: Rviz enables users to configure visualization settings, such as changing colors or styles, to better suit their needs or preferences.
    4. Plugin system: Rviz supports plugins, which are reusable components that provide spe

In [53]:
def ask(query, 
        temperature=0.7,
        top_p=0.95,
        top_k=40,
        max_new_tokens=512,
        format_answer_text=True, 
        return_answer_only=True):
    """
    Takes a query, finds relevant resources/context and generates an answer to the query based on the relevant resources.
    """
    
    # Get just the scores and indices of top related results
    # scores, indices = retrieve_relevant_resources(query=query,
    #                                               embeddings=embeddings)
    search_results = qdrant.search(
    collection_name=my_collection,
    query_vector=model.encode(query),
    limit=5
)
    
    # Create a list of context items
    # context_items = [pages_and_chunks[i] for i in indices]
    context_items = []
    for result in search_results:
        context_items.append({"sentence_chunk": result.payload['text']})

    scores = [result.score for result in search_results]

    # Add score to context item
    for i, item in enumerate(context_items):
        item["score"] = scores[i] # return score back to CPU 
        
    # Format the prompt with context items
    prompt = prompt_formatter(query=query,
                              context_items=context_items)
    
    # Tokenize the prompt
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate an output of tokens
    outputs = llm_model.generate(**input_ids,
                                 temperature=temperature,
                                 do_sample=True,
                                 top_p=top_p,
                                 top_k=top_k,
                                 max_new_tokens=max_new_tokens)
    
    # Turn the output tokens into text
    output_text = tokenizer.decode(outputs[0])

    if format_answer_text:
        # Replace special tokens and unnecessary help message
        prompt_format_matched = prompt.replace('<|im_start|>', '<|im_start|> ').replace('<|im_end|>\n', '<|im_end|> \n')
        output_text = output_text.replace('<s>', '').replace(prompt_format_matched, '').replace('<|im_end|>', '')

    # Only return the answer without the context items
    if return_answer_only:
        return output_text, "No context items returned"
    
    return output_text, context_items

In [54]:
# Define helper function to print wrapped text 
import textwrap

def print_wrapped(text, wrap_length=80):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)

In [55]:
query = random.choice(query_list)
print(f"Query: {query}")

# Answer query with context and return context 
answer, context_items = ask(query=query, 
                            temperature=0.7,
                            max_new_tokens=512,
                            return_answer_only=False)



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Query: WWhat are the main differences between ROS 1 and ROS 2?


In [56]:
print(f"Answer:\n")
print_wrapped(answer)
print(f"Context items:")
context_items

Answer:

     ROS 1 and ROS 2 are two major versions of the Robot Operating System (ROS).
They share some similarities but have key differences. Here are the main
differences between ROS 1 and ROS 2:      1. Language Support: ROS 1 primarily
uses Python for communication between nodes, while ROS 2 supports multiple
languages, including C++, Python, and others, making it more language-agnostic.
2. Data Transport: ROS 1 uses a custom data transport protocol called TCPROS,
while ROS 2 uses the Data Distribution Service (DDS) as its data transport
mechanism. DDS is more reliable, scalable, and real-time capable.     3.
Architecture: ROS 2 has a modular architecture with a smaller core set of
components, making it easier to maintain and extend. ROS 1 has a larger core set
of components.     4. Lifecycle Nodes: ROS 2 introduces a node lifecycle model,
allowing nodes to transition between states (e.g., idle, pre-operational,
operational, and post-operational) and handle events, making the sys

[{'sentence_chunk': "and great libraries that's not all. ros is said to be language agnostic. it means that you can program some parts. of your application. in a programming language and another. part",
  'score': 0.5012295246124268},
 {'sentence_chunk': "powered by ros. using python and c plus plus. ros is also open source with an. active and growing community so. you can directly see what's going on you. can get help",
  'score': 0.41342034935951233},
 {'sentence_chunk': "so this is the why behind ross. now when to use ros maybe. you've already started to program robots. with. an arduino board with a custom code. on a computer and what you will quickly",
  'score': 0.4084434509277344},
 {'sentence_chunk': "and a framework built especially for. robotics applications rao's goal. is to provide a standard for robotic. software. that developers can use and reuse in. any robot once you know how to use ros",
  'score': 0.28597965836524963},
 {'sentence_chunk': 'joystick and so on. and each 